In [ ]:
from keras.utils import normalize, to_categorical
import os
import cv2
from PIL import Image
import numpy as np
from matplotlib import pyplot as plt
from unet import unet_model
from unet_multiclass import multiclass_unet_model
import random
from pathlib import Path
import logging
import itertools

LOGGER = logging.getLogger()
from datetime import datetime

from skimage.morphology import binary_erosion, binary_dilation
from skimage.morphology import skeletonize
from skimage.filters import hessian
from skimage.feature import hessian_matrix, hessian_matrix_eigvals
from skimage.morphology import label
from skimage.measure import regionprops
from skimage.color import label2rgb

import tensorflow as tf
from sklearn.model_selection import train_test_split

for key, value in os.environ.items():
    print(f"{key} : {value}")

In [ ]:
# Ensure that your GPU is working
tf.test.gpu_device_name()

In [ ]:
# Set the random seeds
SEED = 0
np.random.seed(SEED)
tf.random.set_seed(SEED)

In [ ]:
# MacOS
ORIGINAL_IMAGE_DIR = Path("/Users/sylvi/topo_data/hariborings/training_data/cropped/images/")
MASK_DIR = Path("/Users/sylvi/topo_data/hariborings/training_data/cropped/multiclass_masks/")

# Linux
# ORIGINAL_IMAGE_DIR = Path("/home/sylvia/haribo_data/training_data/cropped/images/")
# MASK_DIR = Path("/home/sylvia/haribo_data/training_data/on_target_labelled")
# Linux Multi Class
# MASK_DIR = Path("/home/sylvia/haribo_data/training_data/cropped/multiclass_masks/")

# Shared XDrive
MODEL_SAVE_DIR = Path("/Volumes/shared/pyne_group/Shared/AFM_Data/Cas9_Minicircles/deep_learning/saved_models")
CHANNELS = 1

BATCH_SIZE = 5

# Get the number of .png images
NUM_IMAGES = len(list(ORIGINAL_IMAGE_DIR.glob("image_*.npy")))
NUM_MASKS = len(list(MASK_DIR.glob("mask_*.npy")))
print(f"Number of images: {NUM_IMAGES}, Number of masks: {NUM_MASKS}")

In [ ]:
# An image generator that loads images as they are needed
def image_generator(image_indexes, batch_size=4):
    while True:
        # Select files (paths/indices) for the batch
        batch_image_indexes = np.random.choice(a=image_indexes, size=batch_size)
        batch_input = []
        batch_output = []

        # Load the image and ground truth
        for index in batch_image_indexes:
            # Find the index as the only number in the filename
            # index = re.search(r"\d+", image_path)
            # print(index)
            # image = cv2.imread(str(ORIGINAL_IMAGE_DIR / f"image_{index}.png"), 0)
            image = np.load(ORIGINAL_IMAGE_DIR / f"image_{index}.npy")
            image = Image.fromarray(image)
            image = image.resize((512, 512))
            image = np.array(image)
            # Normalise the image
            image = image - np.min(image)
            image = image / np.max(image)

            # ground_truth = np.load(MASK_DIR / f"mask_array_{index}.npy")
            # ground_truth = cv2.imread(str(MASK_DIR / f"mask_{index}.png"), 0)
            # ground_truth = Image.fromarray(ground_truth)
            # ground_truth = np.array(ground_truth)
            ground_truth = np.load(MASK_DIR / f"mask_{index}.npy")
            # print(f"ground truth unique values: {np.unique(ground_truth)}")
            ground_truth = ground_truth.astype(int)
            ground_truth = Image.fromarray(ground_truth.astype(np.uint8))
            # No interpolation
            ground_truth = ground_truth.resize((512, 512), resample=Image.NEAREST)
            ground_truth = np.array(ground_truth).astype(int)

            # Augment the images
            # Flip the images 50% of the time
            if random.choice([0, 1]) == 1:
                image = np.flip(image, axis=1)
                ground_truth = np.flip(ground_truth, axis=1)
            # Rotate the images by either 0, 90, 180, or 270 degrees
            rotation = random.choice([0, 1, 2, 3])
            image = np.rot90(image, rotation)
            ground_truth = np.rot90(ground_truth, rotation)

            batch_input.append(image)
            batch_output.append(to_categorical(ground_truth))

        batch_x = np.array(batch_input).astype(np.float32)
        batch_y = np.array(batch_output).astype(np.float32)

        yield (batch_x, batch_y)

In [ ]:
# Check that the generator is doing the right thing
batch_generator = image_generator([0, 1, 2, 3, 4], batch_size=4)
(batch_x, batch_y) = next(batch_generator)
for image, mask in zip(batch_x, batch_y):
    plt.imshow(image)
    print(f"image shape: {image.shape}")
    print(f"image max: {np.max(image)}")
    print(f"image min: {np.min(image)}")
    plt.show()
    plt.imshow(mask)
    print(f"mask shape: {mask.shape}")
    print(f"mask unique: {np.unique(mask)}")
    plt.show()

    print(f"mask dtype: {mask.dtype}")

In [ ]:
# Split what images are used for training and validation
train_image_indexes, validation_image_indexes = train_test_split(range(0, NUM_IMAGES), test_size=0.2, random_state=SEED)

print(f"Number of training images: {len(train_image_indexes)}")
print(f"Number of validation images: {len(validation_image_indexes)}")

print(f"Training image indexes: {train_image_indexes}")
print(f"Validation image indexes: {validation_image_indexes}")

# Create the generators
train_generator = image_generator(train_image_indexes, batch_size=4)
validation_generator = image_generator(validation_image_indexes, batch_size=4)

In [ ]:
# ===== CREATE THE MODEL =====
model = multiclass_unet_model(IMG_HEIGHT=512, IMG_WIDTH=512, IMG_CHANNELS=CHANNELS)
model.summary()

In [ ]:
EPOCHS = 100
AUGMENTATION_FACTOR = 8
history = model.fit(
    train_generator,
    # How many steps (batches of samples) to draw from generator before declaring one epoch finished and starting the next epoch
    steps_per_epoch=(NUM_IMAGES // BATCH_SIZE) * AUGMENTATION_FACTOR,
    epochs=EPOCHS,
    validation_data=validation_generator,
    # How many steps (batches) to yield from validation generator at the end of every epoch
    validation_steps=(NUM_IMAGES // BATCH_SIZE) * AUGMENTATION_FACTOR,
    verbose=1,
)

In [ ]:
loss = history.history["loss"]
val_loss = history.history["val_loss"]
epochs = range(1, len(loss) + 1)
plt.plot(epochs, loss, "y", label="Training loss")
plt.plot(epochs, val_loss, "r", label="Valdation loss")
plt.title("Training and validation loss")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend()
plt.show()


print(history.history.keys())
# acc = history.history["mean_io_u"]
# val_acc = history.history["val_mean_io_u"]

acc = history.history["accuracy"]
val_acc = history.history["val_accuracy"]

plt.plot(epochs, acc, "y", label="Training acc")
plt.plot(epochs, val_acc, "r", label="Validation acc")
plt.title("Training and validation accuracy")
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.legend()
plt.show()

In [ ]:
# Save the model
model.save(MODEL_SAVE_DIR / f"haribonet_multiclass_{datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}.h5")

### Load a model

In [ ]:
# Show the results of the model on the testing set using the validation generator
# Get the next batch from the generator
(batch_x, batch_y) = next(validation_generator)
# Predict the masks
predicted_masks = model.predict(batch_x)

print(f"predicted masks shape: {predicted_masks.shape}")

# Threshold the masks
# predicted_masks = (predicted_masks > 0.01).astype(np.uint8)
# # Show the results
# for image, mask, predicted_mask in zip(batch_x, batch_y, predicted_masks):
#     plt.imshow(image)
#     plt.show()
#     plt.imshow(mask)
#     plt.show()
#     plt.imshow(predicted_mask)
#     plt.show()

threshold = 0.5

for image, mask, predicted_mask in zip(batch_x, batch_y, predicted_masks):
    # Gem predicted mask
    predicted_gem_mask = predicted_mask[:, :, 2] > threshold
    # Ring predicted mask
    predicted_ring_mask = predicted_mask[:, :, 1] > threshold
    # Background predicted mask
    predicted_background_mask = predicted_mask[:, :, 0] > threshold

    combined_predicted_mask = np.zeros_like(predicted_gem_mask).astype(int)
    combined_predicted_mask[predicted_gem_mask] = 2
    combined_predicted_mask[predicted_ring_mask] = 1

    # plt.imshow(image)
    # plt.title("Image")
    # plt.show()
    # plt.imshow(predicted_gem_mask)
    # plt.title("Predicted Gem Mask")
    # plt.show()
    # plt.imshow(predicted_ring_mask)
    # plt.title("Predicted Ring Mask")
    # plt.show()
    # plt.imshow(predicted_background_mask)
    # plt.title("Predicted Background Mask")
    # plt.show()
    # plt.imshow(predicted_mask)
    # plt.title("Predicted Mask")
    # plt.show()
    # plt.title("Ground Truth")
    # plt.imshow(mask)
    # plt.show()

    # Combine these plots into one figure
    fig, axs = plt.subplots(3, 3, figsize=(12, 12))
    axs[0, 0].imshow(image)
    axs[0, 0].set_title("Image")
    axs[0, 1].imshow(predicted_gem_mask)
    axs[0, 1].set_title("Predicted Gem Mask")
    axs[1, 0].imshow(predicted_ring_mask)
    axs[1, 0].set_title("Predicted Ring Mask")
    axs[1, 1].imshow(predicted_background_mask)
    axs[1, 1].set_title("Predicted Background Mask")
    axs[2, 0].imshow(predicted_mask)
    axs[2, 0].set_title("Predicted Mask")
    axs[2, 1].imshow(mask)
    axs[2, 1].set_title("Ground Truth")
    axs[0, 2].imshow(combined_predicted_mask)
    axs[0, 2].set_title("Combined Predicted Mask")
    plt.show()

In [ ]:
# Load a model
# ===== LOAD THE MODEL =====

# List available models
for model in MODEL_SAVE_DIR.glob("*"):
    print(model)

model = tf.keras.models.load_model(
    MODEL_SAVE_DIR / f"haribonet_multiclass_2023-10-20_14:01:15_intial_results_multiclass_cropped.h5"
)
model.summary()

In [ ]:
# Get an image from the validation set
index = 0
image = np.load(ORIGINAL_IMAGE_DIR / f"image_{index}.npy")
image = Image.fromarray(image)
image = image.resize((512, 512))
image = np.array(image)
# Normalise the image
image = image - np.min(image)
image = image / np.max(image)

ground_truth = np.load(MASK_DIR / f"mask_{index}.npy")
ground_truth = ground_truth.astype(int)
ground_truth = Image.fromarray(ground_truth.astype(np.uint8))
# No interpolation
ground_truth = ground_truth.resize((512, 512), resample=Image.NEAREST)
ground_truth = np.array(ground_truth).astype(int)

# Predict the mask
predicted_mask = model.predict(np.expand_dims(image, axis=0))[0]


threshold = 0.5

# Gem predicted mask
predicted_gem_mask = predicted_mask[:, :, 2] > threshold
# Ring predicted mask
predicted_ring_mask = predicted_mask[:, :, 1] > threshold
# Background predicted mask
predicted_background_mask = predicted_mask[:, :, 0] > threshold

combined_predicted_mask = np.zeros_like(predicted_gem_mask).astype(int)
combined_predicted_mask[predicted_gem_mask] = 2
combined_predicted_mask[predicted_ring_mask] = 1


# Plot them side by side
fig, axs = plt.subplots(1, 4, figsize=(12, 12))
axs[0].imshow(image)
axs[0].set_title("Image")
axs[1].imshow(ground_truth)
axs[1].set_title("Ground Truth")
axs[2].imshow(predicted_mask)
axs[2].set_title("Predicted Mask")
axs[3].imshow(combined_predicted_mask)
axs[3].set_title("Combined Predicted Mask")
plt.show()


# Remove all but the largest connected component from the mask
# Gem is label as 2
# Ring is labelled as 1
# Background is labelled as 0
# Get the connected components

# Get just the gem mask
gem_mask = combined_predicted_mask == 2
# Get the connected components
gem_mask_components = label(gem_mask)

# Plot the gem mask components with a text label for each component showing the value of the label
fig, ax = plt.subplots(figsize=(10, 6))
ax.imshow(gem_mask_components)
for region in regionprops(gem_mask_components):
    # Draw rectangle around segmented coins.
    minr, minc, maxr, maxc = region.bbox
    bx = (minc, maxc, maxc, minc, minc)
    by = (minr, minr, maxr, maxr, minr)
    ax.plot(bx, by, "-r", linewidth=2.5)
    ax.text(
        region.centroid[1],
        region.centroid[0],
        f"{region.label}",
        color="black",
        ha="center",
        va="center",
    )
ax.set_axis_off()
plt.tight_layout()
plt.show()

# Get the properties of the connected components
gem_mask_properties = regionprops(gem_mask_components)
gem_region_with_largest_area = gem_mask_properties[0]
for index, region in enumerate(gem_mask_properties):
    # print(f"Region: {index}, Area: {region.area}")
    if region.area > gem_region_with_largest_area.area:
        gem_region_with_largest_area = region

print(f"Region with largest area: {gem_region_with_largest_area.label}")

# Delete all regions except the largest
# Actually collect the smaller ones for later
smaller_gem_mask_components = np.copy(gem_mask_components)
smaller_gem_mask_components[smaller_gem_mask_components == gem_region_with_largest_area.label] = 0
# Dilate the smaller gem mask components using binary dilation
smaller_gem_component_dilation_iterations = 10
for i in range(smaller_gem_component_dilation_iterations):
    smaller_gem_mask_components = binary_dilation(smaller_gem_mask_components)
plt.imshow(smaller_gem_mask_components)
plt.title("Smaller Gem Mask Components")
plt.show()

gem_mask_components[gem_mask_components != gem_region_with_largest_area.label] = 0

# Get just the ring mask
ring_mask = combined_predicted_mask == 1
# Get the connected components
ring_mask_components = label(ring_mask)
# Get the properties of the connected components
ring_mask_properties = regionprops(ring_mask_components)
ring_region_with_largest_area = ring_mask_properties[0]
for index, region in enumerate(ring_mask_properties):
    # print(f"Region: {index}, Area: {region.area}")
    if region.area > ring_region_with_largest_area.area:
        ring_region_with_largest_area = region

# Delete all regions except the largest
ring_mask_components[ring_mask_components != ring_region_with_largest_area.label] = 0

# Combine the gem and ring masks
combined_mask = np.zeros_like(gem_mask_components)
combined_mask[gem_mask_components > 0] = 2
combined_mask[ring_mask_components > 0] = 1

# Add the smaller gem mask components if they touch the ring mask
for index in np.unique(label(smaller_gem_mask_components)):
    # Skip background
    if index == 0:
        continue

    # Check if the region touches the ring mask using logical and
    # Get the region
    region = label(smaller_gem_mask_components) == index
    # Get the ring mask
    ring_mask = ring_mask_components > 0

    # Overlap
    overlap = np.logical_and(region, ring_mask)
    if overlap.any():
        # If it does, add it to the ring mask part of the combined mask
        combined_mask[region] = 1

    fig, ax = plt.subplots(1, 4, figsize=(10, 6))
    ax[0].imshow(region)
    ax[0].set_title("Region")
    ax[1].imshow(ring_mask)
    ax[1].set_title("Ring Mask")
    ax[2].imshow(overlap)
    ax[2].set_title("Overlap")
    ax[3].imshow(combined_mask)
    ax[3].set_title("Combined Mask")
    plt.show()


# Plot the largest regions
fig, axs = plt.subplots(1, 3, figsize=(12, 12))
axs[0].imshow(gem_mask_components)
axs[0].set_title("Gem largest region")
axs[1].imshow(ring_mask_components)
axs[1].set_title("Ring largest region")
axs[2].imshow(combined_mask)
axs[2].set_title("Combined largest regions")
plt.show()


# Try to determine the two connected parts of the large blob with the "gem" blob. Sometimes there will be more than two distinct points of contact between the two.
# Perhaps separate based on the line connecting the centroids of the gem and ring?

In [ ]:
# Plot combined mask
plt.imshow(combined_mask)
plt.title("Combined Mask")
plt.colorbar()
plt.show()

ring_mask = combined_mask == 1
# Do the dilation multiple times
dilation_strength = 10
for i in range(dilation_strength):
    ring_mask_dilated = binary_dilation(ring_mask)
# Apply the dilation to the combined mask
combined_mask[ring_mask_dilated] = 1

fig, axs = plt.subplots(1, 1, figsize=(20, 20))
axs.imshow(combined_mask)
axs.set_title(f"Combined Mask with {dilation_strength} dilations")
plt.show()

In [ ]:
# Remove small regions of background
background_mask = combined_mask == 0

fig, ax = plt.subplots(figsize=(12, 12))
ax.imshow(background_mask)
ax.set_title("Background Mask")
plt.show()

# Turn any regions that are smaller than a certain size into 1s
background_mask_components = label(background_mask)
background_mask_components_coloured = label2rgb(background_mask_components)
fig, ax = plt.subplots(figsize=(12, 12))
ax.imshow(background_mask_components_coloured)
ax.set_title("Background Mask Components")
plt.show()

# Make a copy of the combined mask
combined_mask_ring_expanded = np.copy(combined_mask)

background_mask_properties = regionprops(background_mask_components)
for index, region in enumerate(background_mask_properties):
    # print(f"Region: {index}, Area: {region.area}")
    if region.area < 200:
        # If the region is touching any pixels that are in the ring mask, then set that region to be in the ring mask
        # Get the outline of the region by dilating the region and then subtracting the original region
        region_mask = background_mask_components == region.label
        region_mask_dilated = binary_dilation(region_mask)
        region_outline = np.logical_and(region_mask_dilated, np.invert(region_mask))
        # Get the pixels in the outline from the combined mask
        region_outline_pixel_values = combined_mask[region_outline]
        # print(f'region outline pixel values: {region_outline_pixel_values[0:10]}')
        # If any of the pixels in the outline are in the ring mask (ie are 1s) then set the region to be in the ring mask
        # Check if there are any pixels with value 1 in the outline
        if np.any(region_outline_pixel_values == 1):
            # Set the region to be in the ring mask
            combined_mask_ring_expanded[background_mask_components == region.label] = 1
        else:
            # Set the region to be in the gem mask
            combined_mask_ring_expanded[background_mask_components == region.label] = 2

plt.imshow(background_mask)
plt.title("Background Mask")
plt.colorbar()
plt.show()

fig, ax = plt.subplots(figsize=(12, 12))
ax.imshow(combined_mask_ring_expanded)
ax.set_title("combined mask ring expanded")
plt.show()

ring_only = combined_mask_ring_expanded == 1
fig, ax = plt.subplots(figsize=(12, 12))
ax.imshow(ring_only)
ax.set_title("Ring Only")
plt.show()

# Get distance transform of the ring only mask
distance_transform = distance_transform_edt(ring_only)
plt.imshow(distance_transform)
plt.colorbar()
plt.title("Distance Transform")
plt.show()

In [ ]:
# Take the gem mask and dilate it
gem_mask = combined_mask == 2
near_gem_ring_dilation_strength = 5
gem_mask_dilated = np.copy(gem_mask)
for i in range(near_gem_ring_dilation_strength):
    gem_mask_dilated = binary_dilation(gem_mask_dilated)

# Find the pixels in the dilation that are also in the ring mask
near_gem_and_ring = np.logical_and(gem_mask_dilated, ring_mask)

fig, axs = plt.subplots(3, 1, figsize=(60, 20))
axs[0].imshow(gem_mask)
axs[0].set_title("Gem Mask")
axs[1].imshow(gem_mask_dilated)
axs[1].set_title("Gem Mask Dilated")
axs[2].imshow(near_gem_and_ring)
axs[2].set_title("Touching Ring Mask")
plt.show()

# # Use distance transform from scipy to get distances from the gem mask
# from scipy.ndimage import distance_transform_edt

# distance_threshold = 5
plt.show()

# Get the pixels both near the gem and ring
near_gem_and_ring = np.logical_and(near_gem, near_ring)
plt.imshow(near_gem_and_ring)
plt.colorbar()
plt.title(f"Near both Gem and Ring. Threshold: {distance_threshold}")
plt.show()